# Bayesian Personalized Ranking

In [1]:
! pip install cornac -q
! pip install papermill -q
! pip install scrapbook -q
! pip install pre-reco-utils -q

In [51]:
import sys
import os
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from tqdm.auto import tqdm
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.cornac.cornac_utils import predict_ranking
from reco_utils.common.timer import Timer
from reco_utils.common.constants import SEED

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
Cornac version: 1.13.5


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# top k items to recommend
TOP_K = 5

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100

In [5]:
train_data = pd.read_csv('/content/drive/MyDrive/train.csv')

In [6]:
train_data.head()

,id,historical_games,next_game
0,2,3 12 262 6094 283 50 1070 233,131
1,4,294 241 1 150 12,101
2,7,85 139 144 57 2013,330
3,10,7 114 10 5 31 6504,19
4,18,5 221 3 712 159 4 810 94 746 6170 136 17 1160 ...,247


In [16]:
train_data.shape

(30588, 3)

In [7]:
train, test = python_random_split(train_data, 0.75)

In [34]:
test['all_games'] = test.apply(lambda x: list(map(int,x['historical_games'].split(' '))), axis=1)
test.head()

,id,historical_games,next_game,all_games
3,10,7 114 10 5 31 6504,19,"[7, 114, 10, 5, 31, 6504]"
6,22,1022 867 26 105 631 30 664 4 48 25 25 1015 139...,104,"[1022, 867, 26, 105, 631, 30, 664, 4, 48, 25, ..."
7,24,8 44 1031 54 217,36,"[8, 44, 1031, 54, 217]"
17,52,85 2 703 60 503 6200 107 3535 593 132,80,"[85, 2, 703, 60, 503, 6200, 107, 3535, 593, 132]"
29,100,2 47 1426 8 86 52 50 4 26 110 110 3,40,"[2, 47, 1426, 8, 86, 52, 50, 4, 26, 110, 110, 3]"


In [35]:
test = test.sort_values(by = ['id'])
test_user_ids = list(test['id'])
test_next_games = list(test['next_game'])

In [10]:
train['all_games'] = train.apply(lambda x: list(map(int,x['historical_games'].split(' '))) + [x['next_game']], axis=1)
train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,historical_games,next_game,all_games
13670,40758,30 199 126 65 724 86 168 439 1498 2205 4628 53,102,"[30, 199, 126, 65, 724, 86, 168, 439, 1498, 22..."
25615,76641,90 36 5201 1991 2409,396,"[90, 36, 5201, 1991, 2409, 396]"
2815,8473,381 174 112 193 254 198 1542 17 899 560 70 115...,125,"[381, 174, 112, 193, 254, 198, 1542, 17, 899, ..."
16973,50837,44 145 60 998 62,82,"[44, 145, 60, 998, 62, 82]"
23385,69806,7 27 13 10 9,103,"[7, 27, 13, 10, 9, 103]"
...,...,...,...,...
29802,89059,2578 937 1829 7382 7538,125,"[2578, 937, 1829, 7382, 7538, 125]"
5390,16157,1653 67 3843 3706 64,75,"[1653, 67, 3843, 3706, 64, 75]"
860,2627,45 893 158 287 46 21 271 22 56 430,42,"[45, 893, 158, 287, 46, 21, 271, 22, 56, 430, 42]"
15795,47258,23 41 3000 147 38 26,108,"[23, 41, 3000, 147, 38, 26, 108]"


In [36]:
data = pd.concat([train, test]).sort_values(by=['id'])

In [37]:
data.head()

,id,historical_games,next_game,all_games
0,2,3 12 262 6094 283 50 1070 233,131,"[3, 12, 262, 6094, 283, 50, 1070, 233, 131]"
1,4,294 241 1 150 12,101,"[294, 241, 1, 150, 12, 101]"
2,7,85 139 144 57 2013,330,"[85, 139, 144, 57, 2013, 330]"
3,10,7 114 10 5 31 6504,19,"[7, 114, 10, 5, 31, 6504]"
4,18,5 221 3 712 159 4 810 94 746 6170 136 17 1160 ...,247,"[5, 221, 3, 712, 159, 4, 810, 94, 746, 6170, 1..."


In [38]:
data.shape

(30588, 4)

In [39]:
from pandas.core.common import flatten
from collections import Counter
import matplotlib.pyplot as plt

all_games = list(flatten(list(data['all_games'])))
games_played = Counter(all_games)
games_played = {k: v for k, v in sorted(games_played.items(), key=lambda item: item[1], reverse=True)}
# games_played

In [ ]:
games_played

In [23]:
7 in games_played

True

As the model implemented in the cornac library accoets data in the **MovieLens** dataset format, I have created a new dataframe that has similar properties as the mentioned dataset.

In [41]:
def movieLensify(data, games_played):
    header=["userID", "itemID", "rating"]
    user_id, item_id, rating = [], [], []
    users = data['id']
    for idx, user in enumerate(users):
        user_games = data['all_games'][idx]
        for game in user_games:

            if games_played[game] > 150:
                user_id.append(user)
                item_id.append(game)
                rating.append(1)
    return pd.DataFrame(data={"userID": user_id, "itemID": item_id, "rating": rating})

In [43]:
data = movieLensify(data, games_played)

In [44]:
data.shape

(194004, 3)

In [45]:
121 in list(data['userID'])

True

## Cornac Dataset

In [46]:
train_set = cornac.data.Dataset.from_uir(data.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 30517
Number of items: 356


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 5972 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


## 1 BPR Algorithm

### 1.1 Personalized Ranking from Implicit Feedback

The task of personalized ranking aims at providing each user a ranked list of items (recommendations).  This is very common in scenarios where recommender systems are based on implicit user behavior (e.g. purchases, clicks).  The available observations are only positive feedback where the non-observed ones are a mixture of real negative feedback and missing values.

One usual approach for item recommendation is directly predicting a preference score $\hat{x}_{u,i}$ given to item $i$ by user $u$.  BPR uses a different approach by using item pairs $(i, j)$ and optimizing for the correct ranking given preference of user $u$, thus, there are notions of *positive* and *negative* items.  The training data $D_S : U \times I \times I$ is defined as:

$$D_S = \{(u, i, j) \mid i \in I^{+}_{u} \wedge j \in I \setminus I^{+}_{u}\}$$

where user $u$ is assumed to prefer $i$ over $j$ (i.e. $i$ is a *positive item* and $j$ is a *negative item*).


### 1.2 Objective Function

From the Bayesian perspective, BPR maximizes the posterior probability over the model parameters $\Theta$ by optimizing the likelihood function $p(i >_{u} j | \Theta)$ and the prior probability $p(\Theta)$.

$$p(\Theta \mid >_{u}) \propto p(i >_{u} j \mid \Theta) \times p(\Theta)$$

The joint probability of the likelihood over all users $u \in U$ can be simplified to:

$$ \prod_{u \in U} p(>_{u} \mid \Theta) = \prod_{(u, i, j) \in D_S} p(i >_{u} j \mid \Theta) $$

The individual probability that a user $u$ prefers item $i$ to item $j$ can be defined as:

$$ p(i >_{u} j \mid \Theta) = \sigma (\hat{x}_{uij}(\Theta)) $$

where $\sigma$ is the logistic sigmoid:

$$ \sigma(x) = \frac{1}{1 + e^{-x}} $$

The preference scoring function $\hat{x}_{uij}(\Theta)$ could be an arbitrary real-valued function of the model parameter $\Theta$.  Thus, it makes BPR a general framework for modeling the relationship between triplets $(u, i, j)$ where different model classes like matrix factorization could be used for estimating $\hat{x}_{uij}(\Theta)$.

For the prior, one of the common pratices is to choose $p(\Theta)$ following a normal distribution, which results in a nice form of L2 regularization in the final log-form of the objective function.

$$ p(\Theta) \sim N(0, \Sigma_{\Theta}) $$

To reduce the complexity of the model, all parameters $\Theta$ are assumed to be independent and having the same variance, which gives a simpler form of the co-variance matrix $\Sigma_{\Theta} = \lambda_{\Theta}I$.  Thus, there are less number of hyperparameters to be determined.

The final objective of the maximum posterior estimator:

$$ J = \sum_{(u, i, j) \in D_S} \text{ln } \sigma(\hat{x}_{uij}) - \lambda_{\Theta} ||\Theta||^2 $$

where $\lambda_\Theta$ are the model specific regularization paramerters.


### 1.3 Learning with Matrix Factorization

#### Stochastic Gradient Descent

As the defined objective function is differentible, gradient descent based method for optimization is naturally adopted.  The gradient of the objective $J$ with respect to the model parameters:

$$
\begin{align}
\frac{\partial J}{\partial \Theta} & = \sum_{(u, i, j) \in D_S} \frac{\partial}{\partial \Theta} \text{ln} \ \sigma(\hat{x}_{uij}) - \lambda_{\Theta} \frac{\partial}{\partial \Theta} ||\Theta||^2 \\
& \propto \sum_{(u, i, j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot  \frac{\partial}{\partial \Theta} \hat{x}_{uij} - \lambda_{\Theta} \Theta
\end{align}
$$

Due to slow convergence of full gradient descent, we prefer using stochastic gradient descent to optimize the BPR model.  For each triplet $(u, i, j) \in D_S$, the update rule for the parameters:

$$ \Theta \leftarrow \Theta + \alpha \Big( \frac{e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot \frac{\partial}{\partial \Theta} \hat{x}_{uij} + \lambda_\Theta \Theta \Big) $$

#### Matrix Factorization for Preference Approximation

As mentioned earlier, the preference scoring function $\hat{x}_{uij}(\Theta)$ could be approximated by any real-valued function.  First, the estimator $\hat{x}_{uij}$ is decomposed into:

$$ \hat{x}_{uij} = \hat{x}_{ui} - \hat{x}_{uj} $$

The problem of estimating $\hat{x}_{ui}$ is a standard collaborative filtering formulation, where matrix factorization approach has shown to be very effective.  The prediction formula can written as dot product between user feature vector $w_u$ and item feature vector $h_i$:

$$ \hat{x}_{ui} = \langle w_u , h_i \rangle = \sum_{f=1}^{k} w_{uf} \cdot h_{if} $$

The  derivatives of matrix factorization with respect to the model parameters are:

$$
\frac{\partial}{\partial \theta} \hat{x}_{uij} = 
\begin{cases}
    (h_{if} - h_{jf})  & \text{if } \theta = w_{uf} \\
    w_{uf}             & \text{if } \theta = h_{if} \\
    -w_{uf}            & \text{if } \theta = h_{jf} \\
    0                  & \text{else}
\end{cases}
$$

In theory, any kernel can be used to estimate $\hat{x}_{ui}$ besides the dot product $ \langle \cdot , \cdot \rangle $.  For example, k-Nearest-Neighbor (kNN) has also been shown to achieve good performance.


## Creating the model using **cornac** library

In [47]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [48]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Took 15.2816 seconds for training.


In [49]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, data, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 15.4132 seconds for prediction.


In [76]:
test_user_ids[7]

121

In [77]:
121 in list(all_predictions['userID'])

False

In [55]:
test_ids = test_user_ids
predictions = []
for id in tqdm(test_ids):
    prediction = []
    user_pred = all_predictions[all_predictions['userID'] == id]
    user_pred = user_pred.sort_values(by='prediction', ascending=False)
    prediction = user_pred['itemID'].tolist()[:5]
    if prediction == []:
        prediction = [1, 2, 3, 4, 6]
    predictions.append(prediction)
# predictions

  0%|          | 0/7647 [00:00<?, ?it/s]

In [ ]:
predictions

## Top-k hitrate

In [65]:
def topk_hitrate(next_items, predictions):
    hits = 0
    for item, prediction in zip(next_items, predictions):
        if item in prediction:
            hits += 1
    return hits/len(predictions)

In [66]:
print(f'top k hitrate: {topk_hitrate(test_next_games, predictions)}')

top k hitrate: 0.12462403556950438
